In [ ]:
import os

import dotenv
import fsspec
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

load_dotenv()

# Configure cloud and Dask settings
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}


coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
shorelines_col = coclico_catalog.get_child("shorelinemonitor-shorelines")

In [ ]:
shorelines_extents = read_snapshot(shorelines_col, storage_options=storage_options)

In [ ]:
shorelines_extents.explore()

In [ ]:
import coastpy
sds_engine = coastpy.io.STACQueryEngine(
    stac_collection=shorelines_col,
    storage_backend="azure",
)

In [ ]:
from ipyleaflet import Map, basemaps

m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = -6.228120, 107.820412
m.zoom = 9
m.layout.height = "800px"
m

In [ ]:
# wait a couple of seconds before running this cell
minx, miny, maxx, maxy = m.west, m.south, m.east, m.north
roi = gpd.GeoDataFrame(geometry=[shapely.box(minx, miny, maxx, maxy)], crs=4326)


In [ ]:
shorelines = sds_engine.get_data_within_bbox(minx, miny, maxx, maxy)

In [ ]:
import sys

sys.path.insert(0, "../src")

from coastmonitor.io.drive_config import configure_instance

configure_instance(branch="dev")

import logging
import os
import pathlib

import duckdb
import geopandas as gpd
import holoviews as hv
import hvplot.pandas
import pandas as pd
import panel as pn
import pystac
import shapely
from coastmonitor.geo.geometries import geo_bbox
from coastmonitor.io.load import retrieve_rois
from coastmonitor.query_engine import HREFQueryEngine, STACQueryEngine
from dotenv import load_dotenv

import coastpy

load_dotenv(override=True)

sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
storage_options = {"account_name": account_name, "credential": sas_token}

# Adjust logging level for azure and rasterio
logging.getLogger("azure").setLevel(logging.WARNING)
logging.getLogger("rasterio").setLevel(logging.WARNING)